### import packages

In [1]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity

### Specify directories to data files

In [2]:
r1_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\"
r2_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\"

# lb_dir = "/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif"
# bad_roi='/test/bad_roi_list.npy'
# spot_dir = '/test/LHA3_R3_medium/spots/'
# out_dir = '/test/LHA3_R3_medium/spots/'
# img_dir='/test/LHA3_R3_medium/stitching/export.n5'


fix_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\export_sigma3.n5"
reg_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped"  # directory to the registered image
out_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped\\testout-july29"  # where the output files should be saved 
lb_dir  = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\segmentation\\lt171_gene_4tile_r2-c3.tif"  # directory to the segmentation mask (tif format accepted here)

bad_roi = os.path.join(out_dir, "bad_roi_list.npy")
spot_dir = os.path.join(r2_dir, "spots")
out_dir = spot_dir
img_dir = fix_dir

print(bad_roi)
print(spot_dir)

D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\registration\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\warped\testout-july29\bad_roi_list.npy
D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots


### Load segmentation mask

In [3]:
%%time
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

number of ROIs in total: 13758
number of bad ROIs: 892
CPU times: total: 17.2 s
Wall time: 6.58 s


### Extract ROI properties
The roi_prop function exports the following: 
- ROI ID
- position (x, y, z in um, pre-expansion)
- size (volume, pre-expansion) 
- mionr_axis_length
- major_axis_length
- eccentricity =  mionr_axis_length/major_axis_length
- solidity = ROI volume/ROI convex hull

In [10]:
lb.shape

(1019, 915, 916)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=int32)

In [12]:
%%time
df=roi_prop.roi_prop(lb)

CPU times: total: 10h 34min 18s
Wall time: 32min 42s


### Remove bad ROIs and save the file

In [13]:
%%time
df_filtered=df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi=df_filtered.roi.astype(int)
df_filtered.to_csv(out_dir+'roi.csv')

<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: total: 1.14 s
Wall time: 1.15 s


### Lipofuscin subtraction (optional)

In [19]:
%%time
# Load spot files for lipofuscin subtraction
channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# Identify lipofuscin spots
c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# Remove lipofuscin spots
true_pos_c0 = np.delete(channel_1, pAind, axis=0)
true_pos_c1 = np.delete(channel_2, pBind, axis=0)

# %%time
# # Load spot files for lipofuscin subtraction
# channel_1=np.loadtxt(spot_dir+'/'+'merged_points_c0.txt',delimiter=',')
# channel_2=np.loadtxt(spot_dir+'/'+'merged_points_c1.txt',delimiter=',')
# # Identify lipofuscin spots
# c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# # Remove lipofuscin spots
# true_pos_c0 = np.delete(channel_1, pAind, axis=0)
# true_pos_c1 = np.delete(channel_2, pBind, axis=0)

CPU times: total: 3.73 s
Wall time: 3.72 s


### spot extraction and save file

In [20]:
%%time
spotcount=spot.spot_counts(lb,spot_dir)
spotcount.to_csv(out_dir+'/spots.csv')

Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c0.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c1.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c2.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c4.txt
CPU times: total: 1min 4s
Wall time: 1min 4s


### Intensity measurement and save file

In [21]:
%%time
df_intensity=intensity.measure_intensity(lb, img_dir,'c0')
df_intensity.to_csv(out_dir+'/intennsity_c0.csv')

CPU times: total: 31.1 s
Wall time: 31.3 s


In [22]:
df_intensity

,roi,c0
0,1,237.725932
1,2,92.013229
2,3,92.587272
3,4,91.877853
4,5,92.523482
...,...,...
13753,13754,91.712826
13754,13755,91.752961
13755,13756,91.721380
13756,13757,91.947368
